## 데이터 전처리

- Null값 처리
- 불필요한 피처 제거
- Encoding 등

In [3]:
from sklearn.preprocessing import LabelEncoder

def fillna(df):
    df['Age'].fillna(df['Age'].mean, inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

def encoding(df):
    df['Cabin'] = df['Cabin'].str[0]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        encoder = LabelEncoder()
        encoder = encoder.fit(df[feature])
        df[feature] = encoder.transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = encoding(df)
    return df

데이터 전처리를 수행하는 tranfrom_features() 함수로 원본데이터 가공

In [4]:
# 원본데이터 로드, 피처 데이터 세트와 레이블 데이터 세트 추출
import pandas as pd
import numpy as np
titanic_df = pd.read_csv('titanic_train.csv')
y = titanic_df['Survived']
X = titanic_df.drop(['Survived'],axis=1)
X= transform_features(X)

In [5]:
# train, test 분리
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=11)